In [100]:
import random
import re
import math
import numpy as np
import pandas as pd

In [132]:
pokoleniya=100
population_size=10
chromosome_bodyoncetoldme=52
selection_count=5
mutation_kek=0.1
steps=False

In [83]:
def parse_tsp_file(filename):
    result = []
    with open(filename) as f:
        for line in f:
            m = re.match(r'(\d+) (\d+\.\d+) (\d+\.\d+)', line)
            if m:
                temp=m.groups()[1:]
                result.append((float(temp[0]), float(temp[1])))
    return result

# parse_tsp_file("berlin52.tsp.txt")

In [84]:
def read_optimal(filename):
    result = []
    with open(filename) as f:
        for line in f:
            m = re.match(r'^\d+$', line.rstrip())
            if m:
                result.append(int(m.group()))

    return result

# read_optimal("berlin52.opt.tour.txt")

In [85]:
data=parse_tsp_file("berlin52.tsp.txt")
optimal=read_optimal("berlin52.opt.tour.txt")

In [86]:
def prisposobl(chromosome):
    result=0
    for i in range(0, len(chromosome)):
        a=data[chromosome[i]-1]
        if(i<len(data)-1):
            b=data[chromosome[i+1]-1]
        elif(i==len(data)-1): 
            b=data[chromosome[0]-1]
        result+=math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)
    return result

# prisposobl(optimal)

In [94]:
from operator import itemgetter
def roulette_selection(data, sel_count):
    pool=[]
    sectors=[]
    wheel=[]
    f_sum=0

    for chromosome in data:
        f=1/(1+prisposobl(chromosome))
        f_sum+=f
    for chromosome in data:
        f=1/(1+prisposobl(chromosome))
        v=f/f_sum*100
        sectors.append((v, chromosome))
    sectors=sorted(sectors, key=itemgetter(0))
    point=0
    for i in range(0,len(sectors)-1):
        wheel.append((sectors[i][1], (point, point+sectors[i][0])))
        point+=sectors[i][0]
        if(i==len(sectors)-2): 
            wheel.append((sectors[i][1], (point, 100)))

    for i in range(sel_count):
        choice=random.randint(0,100)
        # print(choice)
        for j in wheel:
            # if((choice>=j[1][0]) and (choice<=j[1][1]) and (j[0] not in pool)): 
            if((choice>=j[1][0]) and (choice<=j[1][1])): 
                pool.append(j[0])
                break

    return pool

# roulette_selection([[1, 49, 32, 45, 19, 41, 8, 9, 10, 43, 33, 51, 11, 52, 14, 13, 47, 26, 27, 28, 12, 25, 4, 6, 15, 5, 24, 48, 38, 37, 40, 39, 36, 35, 34, 44, 46, 16, 29, 50, 20, 23, 30, 2, 7, 42, 21, 17, 3, 18, 31, 22],[1, 49, 32, 45, 19, 41, 8, 9, 10, 43, 33, 51, 11, 52, 14, 13, 47, 26, 27, 28, 12, 25, 4, 6, 15, 5, 24, 48, 38, 35, 40, 39, 36, 37, 34, 44, 46, 16, 29, 50, 20, 23, 30, 2, 7, 42, 21, 17, 3, 18, 31, 22],[6, 49, 32, 45, 19, 41, 8, 9, 10, 43, 33, 51, 11, 52, 14, 13, 47, 26, 27, 28, 12, 25, 4, 1, 15, 5, 24, 48, 38, 30, 16, 39, 36, 35, 34, 44, 46, 40, 29, 50, 20, 23, 37, 2, 7, 42, 21, 17, 3, 18, 31, 22],[1, 49, 32, 45, 19, 41, 8, 9, 10, 43, 33, 51, 11, 52, 14, 13, 47, 26, 27, 28, 12, 25, 31, 6, 15, 5, 24, 48, 38, 37, 40, 39, 36, 35, 34, 44, 46, 16, 29, 50, 20, 23, 30, 2, 7, 42, 21, 17, 3, 18, 4, 22],[1, 49, 32, 45, 19, 41, 8, 9, 10, 43, 40, 51, 48, 52, 14, 13, 47, 26, 27, 28, 7, 25, 31, 6, 15, 5, 24, 11, 38, 37, 33, 39, 36, 35, 34, 44, 46, 16, 29, 50, 20, 23, 30, 2, 12, 42, 21, 17, 3, 18, 4, 22]], 5)

In [76]:
def permutation_crossover(parent1, parent2, chromo_length):
    chromo_length-=2
    par1_start=parent1[0]
    par2_start=parent2[0]
    par1_end=parent1[len(parent1)-1]
    par2_end=parent2[len(parent1)-1]
    parent1=parent1[1:len(parent1)-1]
    parent2=parent2[1:len(parent2)-1]

    pos1=random.randint(1, chromo_length//2-1)
    pos2=random.randint(pos1+1, chromo_length-1)
    middle=parent1[pos1:pos2+1]
    shrek=[]
    for i in parent2:
        if not (i in middle): shrek.append(i)
    child1=[]
    for i in range(0, chromo_length-1):
        if(i<pos1): child1.append(shrek.pop(0))
        elif(i==pos1): child1+=middle
        elif(i==pos2): child1.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)>0): child1.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)==0): child1.append(parent2[i])
    
    middle=parent2[pos1:pos2+1]
    shrek=[]
    for i in parent1:
        if not (i in middle): shrek.append(i)
    child2=[]
    for i in range(0, chromo_length-1):
        if(i<pos1): child2.append(shrek.pop(0))
        elif(i==pos1): child2+=middle
        elif(i==pos2): child2.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)>0): child2.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)==0): child2.append(parent1[i])

    child1.insert(0, par1_start)
    child1.append(par1_end)
    child2.insert(0, par2_start)
    child2.append(par2_end)
    return child1, child2

permutation_crossover([1,2,3,4,5,6,7,8], [8,7,6,5,4,3,2,1], 8)

([1, 7, 6, 4, 5, 3, 2, 8], [8, 2, 3, 5, 4, 6, 7, 1])

In [72]:
def mutation(chromosome):
    result=chromosome[1:len(chromosome)-1]
    choice=random.random()
    if(choice-mutation_kek<=0):
        random.shuffle(result)
    result.insert(0, chromosome[0])
    result.append(chromosome[len(chromosome)-1])
    return result

mutation([1,2,3,4,5,6,7,8])

[1, 6, 5, 3, 2, 7, 4, 8]

In [88]:
def generate_population(population):
    result=[]
    for i in range(population):
        temp=optimal[1:len(optimal)-1]
        random.shuffle(temp)
        temp.insert(0, optimal[0])
        temp.append(optimal[len(optimal)-1])
        result.append(temp)
    return result

generate_population(5)

[[1,
  51,
  2,
  52,
  23,
  11,
  48,
  29,
  30,
  21,
  6,
  5,
  43,
  35,
  25,
  9,
  27,
  13,
  40,
  42,
  10,
  46,
  50,
  34,
  28,
  31,
  49,
  38,
  16,
  44,
  26,
  7,
  36,
  14,
  15,
  45,
  8,
  24,
  20,
  37,
  41,
  17,
  19,
  12,
  4,
  32,
  33,
  3,
  47,
  39,
  18,
  22],
 [1,
  9,
  30,
  21,
  40,
  37,
  34,
  52,
  2,
  47,
  18,
  51,
  17,
  36,
  3,
  31,
  44,
  19,
  27,
  43,
  5,
  15,
  29,
  41,
  13,
  32,
  45,
  10,
  49,
  42,
  35,
  7,
  4,
  11,
  26,
  8,
  6,
  46,
  25,
  20,
  12,
  38,
  48,
  14,
  24,
  16,
  39,
  33,
  23,
  28,
  50,
  22],
 [1,
  34,
  40,
  50,
  48,
  7,
  33,
  52,
  49,
  3,
  14,
  4,
  37,
  13,
  18,
  43,
  27,
  25,
  15,
  5,
  19,
  8,
  30,
  23,
  26,
  10,
  44,
  35,
  31,
  6,
  11,
  47,
  42,
  21,
  28,
  16,
  17,
  51,
  45,
  20,
  2,
  32,
  41,
  38,
  12,
  9,
  29,
  46,
  39,
  24,
  36,
  22],
 [1,
  26,
  51,
  27,
  20,
  40,
  42,
  46,
  50,
  2,
  4,
  34,
  52,
  39,
  30,
 

In [102]:
def print_population(data):
    df=pd.DataFrame(data)
    print(df)

def population_best_result(data):
    result=999999
    best_shrek=[]
    for i in data:
        shrek=prisposobl(i)
        if(shrek<result): 
            result=shrek
            best_shrek=i
    return (result, best_shrek)


In [133]:
if(steps):
    print(1)

else:
    current_population=generate_population(population_size)
    print("первая популяция создана:")
    print_population(current_population)
    print("Лучшее значение приспособленности:")
    print(population_best_result(current_population))
    for pok in range(pokoleniya):
        current_population=roulette_selection(current_population, selection_count)
        print("селекция произведена")
        
        for i in range(selection_count//2):
            parent1=random.randint(0, selection_count-1)
            parent2=random.randint(0, selection_count-1)
            while(parent2==parent1): parent2=random.randint(0, selection_count-1)
            children=permutation_crossover(current_population[parent1], current_population[parent2], chromosome_bodyoncetoldme)
            current_population.append(children[0])
            current_population.append(children[1])
        if(len(current_population)<population_size): 
            current_population.append(generate_population(1)[0])
        print("скрещивание произведено")
        print_population(current_population)

        for chel in current_population:
            chel=mutation(chel)
        print("мутация произведена")
        print(f"популяция в конце {pok+1} поколения:")
        print_population(current_population)
        print("Лучшее значение приспособленности:")
        print(population_best_result(current_population))

первая популяция создана:
   0   1   2   3   4   5   6   7   8   9   ...  42  43  44  45  46  47  48  \
0   1   8  52  35  25   5   4  42  44  40  ...  18  31  38  34  11  32  47   
1   1  23  33  11  34  14  42  52   6  12  ...  46  25  27   5  51  39  48   
2   1  21   2   5  23  34  14   8  47  13  ...   7  46  18  26  45  35  15   
3   1  20   4  38   7  42  14  19  24  39  ...  11  10  36   3  33  27  16   
4   1  17  32  30  37  36  38  27  44  47  ...   8  52  14  12  34  49  19   
5   1  16   9  19  35  37  10   6  42  21  ...  40  25  41  36  49  17  11   
6   1  30   6  41  45  24  19   4  15  18  ...  31  33  50  42  25  10  32   
7   1  43  30  24  40  50  15  51  28  20  ...  26  32  48   6   9  37  25   
8   1   6  36  30  49  45   2  46  14  27  ...  17   3  13  18  43  16  19   
9   1  42   6  47   5  12   8   9  44  19  ...  13  45  30  27  24  40  35   

   49  50  51  
0  30  16  22  
1  45   2  22  
2  28   6  22  
3  21  47  22  
4  25   5  22  
5  38  47  22  
6  